In [1]:
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import pandas as pd
import geopandas as gpd
import folium
from folium.plugins import HeatMap
from folium import FeatureGroup, LayerControl
from branca.colormap import linear
import branca.colormap as cm

import pandas_scripts.excel_scripts as exs
import pandas_scripts.locatie_coords_geoshape_script as lcs

import json



In [2]:
DB_USER = "s1144085"
DB_PASS = "9515"
DB_HOST = "95.217.3.61"
DB_PORT = "5432"
DB_NAME = "minor_s1144085"
SCHEMA_NAME = "INMINDAA_project"

In [3]:
DATABASE_URL = f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(DATABASE_URL)

In [4]:
def fetch_leefbarometer_data():
    query = f"""
    SELECT *
    FROM "{SCHEMA_NAME}".{"dim_leefbarometer"}
    """
    return pd.read_sql(query, con=engine)

In [5]:
def fetch_armoede_data():
    query = f"""
    SELECT *
    FROM "{SCHEMA_NAME}".{"dim_armoede"}
    """
    return pd.read_sql(query, con=engine)

In [6]:
df_leef = fetch_leefbarometer_data()
df_armoede = fetch_armoede_data()

te_verwijderen_score = [2.9613476]
df_leef = df_leef[~df_leef['lbm_score'].isin(te_verwijderen_score)]

geodf = lcs.get_geodataframe()

df_leef['pc4_code'] = df_leef['pc4_code'].astype(str)
df_armoede['pc4_code'] = df_armoede['pc4_code'].astype(str)
geodf['pc4_code'] = geodf['pc4_code'].astype(str)

geodf_merged = geodf.merge(df_leef, on='pc4_code', how='left')
geodf_merged = geodf_merged.merge(df_armoede, on='pc4_code', how='left')
geodf_merged = geodf_merged.dropna(subset=["lbm_score", "g_ink_pi"])

Empty DataFrame
Columns: [pc4_code, Geo Shape]
Index: []


In [7]:
# colors = ["black", "lightblue"]
# my_cmap = LinearSegmentedColormap.from_list("custom_cmap", colors, N=10)
#
# # Color options: 'hsv' 'cool'
# fig, ax = plt.subplots(figsize=(8, 8))
#
# ax.set_title('Leefbaarometer-score in Nederland')
# plt.style.use('dark_background')
# ax.set_facecolor('black')
# # plt.axis('off')
# geodf_merged.plot(
#     column='lbm_score',
#     cmap=my_cmap,
#     legend=True,
#     ax=ax
# )
#
# #54 50 3.2 7.5
#
# plt.show()

In [ ]:
# colormap_lbm = linear.YlGn_09.scale(
#     geodf_merged.lbm_score.min(), geodf_merged.lbm_score.max()
# )
# lbm_dict = geodf_merged.set_index("pc4_code")["lbm_score"]
#
# colormap_ink = linear.YlGn_09.scale(
#     geodf_merged.g_ink_pi.min(), geodf_merged.g_ink_pi.max()
# )
# ink_dict = geodf_merged.set_index("pc4_code")["g_ink_pi"]

linear1 = cm.LinearColormap(["green", "yellow", "red"], vmin=3, vmax=5)
linear2 = cm.LinearColormap(["green", "yellow", "red"], vmin=0, vmax=500000)

nl_map = folium.Map(location=[52.132633, 5.291266], zoom_start=8, tiles='Cartodb Positron')

lbm_layer = folium.FeatureGroup(name="Leefbaarometer", show=True)
folium.GeoJson(
    geodf_merged,
    style_function=lambda feature: {
        "fillColor": linear1, # colormap_lbm(lbm_dict[feature["properties"]["pc4_code"]]),
        "color": "black",
        "weight": 0.2,
        "fillOpacity": 0.7,
    },
    # tooltip=folium.GeoJsonTooltip(fields=["pc4_code", "lbm_score"]),
).add_to(lbm_layer)
lbm_layer.add_to(nl_map)
# colormap_lbm.caption = "Leefbaarometer Score"
# colormap_lbm.add_to(nl_map)

ink_layer = folium.FeatureGroup(name="Inkomensscore", show=False)
folium.GeoJson(
    geodf_merged,
    style_function=lambda feature: {
        "fillColor": linear2,# colormap_ink(ink_dict[feature["properties"]["pc4_code"]]),
        "color": "black",
        "weight": 0.2,
        "fillOpacity": 0.7,
    },
    # tooltip=folium.GeoJsonTooltip(fields=["pc4_code", "g_ink_pi"]),
).add_to(ink_layer)
ink_layer.add_to(nl_map)
# colormap_ink.caption = "Inkomensscore"
# colormap_ink.add_to(nl_map)

folium.LayerControl(collapsed=False).add_to(nl_map)
display(nl_map)

# nl_map.save("leefbaarometer_en_inkomens_heatmap.html")